## DIA 022: Optimización de Rendimiento y Escalabilidad de la API de Predicción

1. Objetivos del Día 22
Optimizar el Rendimiento de la API:

Implementar mecanismos de caching para reducir la carga del servidor y disminuir la latencia.
Optimizar el modelo de machine learning para mejorar los tiempos de inferencia.
Configurar la Escalabilidad en Heroku:

Ajustar la configuración de dynos para manejar incrementos en el tráfico.
Implementar auto-scaling si la plataforma lo permite.
Mejorar la Gestión de Logs y Monitoreo:

Refinar las herramientas de logging y monitoreo para una mejor visibilidad del rendimiento de la API.
2. Optimizar el Rendimiento de la API
2.1. Implementar Caching con Redis
El caching almacena las respuestas a solicitudes frecuentes, reduciendo la necesidad de recalcular predicciones y disminuyendo la latencia.

2.1.1. Instalar Redis
Agregar Redis a requirements.txt:

plaintext
Copiar
redis==4.5.5
Actualizar las Dependencias:

bash
Copiar
pip install -r requirements.txt
Configurar Redis en Heroku:

Añadir el Add-on de Redis:
bash
Copiar
heroku addons:create heroku-redis:hobby-dev --app mnist-flask-api
Obtener la URL de Redis:
bash
Copiar
heroku config:get REDIS_URL --app mnist-flask-api
2.1.2. Integrar Redis en api.py
Actualiza tu archivo api.py para utilizar Redis como sistema de caching.

python
Copiar
# api.py

from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
import numpy as np
import io
import os
import redis
import hashlib
from functools import wraps

app = Flask(__name__)

# Configurar Redis
redis_url = os.getenv('REDIS_URL', 'redis://localhost:6379/0')
cache = redis.Redis.from_url(redis_url)

# Cargar el modelo optimizado
modelo_web = tf.keras.models.load_model('modelo_cnn_transfer_learning_mnist_final_optimizado.h5')

# Obtener la API Key desde variables de entorno
API_KEY = os.getenv('API_KEY')

def require_api_key(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        key = request.headers.get('x-api-key')
        if key and key == API_KEY:
            return func(*args, **kwargs)
        else:
            return jsonify({'error': 'Unauthorized access'}), 401
    return wrapper

def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def generate_cache_key(file_bytes):
    return hashlib.sha256(file_bytes).hexdigest()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
@require_api_key
def predict_web():
    if 'file' not in request.files:
        return jsonify({'error': 'No se encontró el archivo en la solicitud.'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No se seleccionó ningún archivo.'}), 400
    if not allowed_file(file.filename):
        return jsonify({'error': 'Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF.'}), 400
    try:
        # Leer los bytes del archivo para generar la clave de cache
        file_bytes = file.read()
        cache_key = generate_cache_key(file_bytes)
        
        # Verificar si la predicción ya está en cache
        cached_result = cache.get(cache_key)
        if cached_result:
            prediccion, probabilidad = cached_result.decode('utf-8').split(',')
            return jsonify({'prediccion': int(prediccion), 'probabilidad': float(probabilidad)})

        # Procesar la imagen
        img = Image.open(io.BytesIO(file_bytes)).convert('L')  # Convertir a escala de grises
        img = img.resize((28, 28))  # Redimensionar a 28x28
        img_array = np.array(img).astype('float32') / 255.0  # Normalizar
        img_array = img_array.reshape((1, 28, 28, 1))  # Añadir dimensión de batch

        # Preprocesar para el modelo (redimensionar a 224x224 y convertir a RGB)
        img_resized = tf.image.resize(img_array, [224, 224])
        img_rgb = tf.image.grayscale_to_rgb(img_resized)

        # Realizar la predicción
        pred = modelo_web.predict(img_rgb)
        pred_class = np.argmax(pred, axis=1)[0]
        pred_prob = float(np.max(pred))

        # Almacenar el resultado en cache por 1 hora
        cache.setex(cache_key, 3600, f"{pred_class},{pred_prob}")

        return jsonify({'prediccion': int(pred_class), 'probabilidad': pred_prob})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)
Explicación:

Integración con Redis:
Conectamos a Redis utilizando la URL proporcionada por Heroku.
Creamos una función generate_cache_key que genera una clave única basada en los bytes del archivo.
Caching de Resultados:
Antes de procesar la imagen, verificamos si la predicción ya está en cache.
Si está en cache, devolvemos el resultado almacenado.
Si no está en cache, procesamos la imagen, realizamos la predicción y almacenamos el resultado en Redis por 1 hora (3600 segundos).
2.2. Optimizar el Modelo para Reducir la Latencia
Reducir el tiempo que tarda el modelo en realizar predicciones puede mejorar significativamente la experiencia del usuario.

2.2.1. Aplicar Quantización al Modelo
La quantización reduce el tamaño del modelo y acelera las inferencias sin una pérdida significativa en la precisión.

Instalar TensorFlow Lite (si no está instalado):

bash
Copiar
pip install tensorflow
Crear un Script para Quantizar el Modelo:

Crea un archivo llamado quantize_model.py en el directorio raíz de tu proyecto.

python
Copiar
# quantize_model.py

import tensorflow as tf

# Cargar el modelo existente
model = tf.keras.models.load_model('modelo_cnn_transfer_learning_mnist_final_optimizado.h5')

# Convertir el modelo a TensorFlow Lite con quantización
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Guardar el modelo quantizado
with open('modelo_cnn_transfer_learning_mnist_quantizado.tflite', 'wb') as f:
    f.write(tflite_quant_model)

print("Modelo quantizado guardado como 'modelo_cnn_transfer_learning_mnist_quantizado.tflite'")
Ejecutar el Script de Quantización:

bash
Copiar
python quantize_model.py
Actualizar api.py para Utilizar el Modelo Quantizado:

Modifica la carga del modelo para usar el modelo quantizado.

python
Copiar
# api.py (modificación en la carga del modelo)

# Cargar el modelo quantizado
interpreter = tf.lite.Interpreter(model_path='modelo_cnn_transfer_learning_mnist_quantizado.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
Modificar la función de predicción:

python
Copiar
# api.py (modificación en la función predict_web)

def predict_web():
    # ... (código anterior)
    try:
        # ... (código de procesamiento de imagen)

        # Realizar la predicción usando TensorFlow Lite
        interpreter.set_tensor(input_details[0]['index'], img_rgb.numpy())
        interpreter.invoke()
        pred = interpreter.get_tensor(output_details[0]['index'])
        pred_class = np.argmax(pred, axis=1)[0]
        pred_prob = float(np.max(pred))

        # Almacenar el resultado en cache por 1 hora
        cache.setex(cache_key, 3600, f"{pred_class},{pred_prob}")

        return jsonify({'prediccion': int(pred_class), 'probabilidad': pred_prob})
    except Exception as e:
        return jsonify({'error': str(e)}), 500
Nota: Asegúrate de que el modelo quantizado sea compatible con TensorFlow Lite y que las dimensiones de entrada sean correctas.

2.2.2. Evaluar el Impacto de la Optimización
Después de aplicar la quantización, es importante evaluar si hubo mejoras en el rendimiento y si la precisión del modelo se mantiene aceptable.

Realizar Pruebas de Rendimiento:

Medir el tiempo de inferencia antes y después de la optimización.
Utilizar herramientas como timeit o perfiles de rendimiento.
Verificar la Precisión del Modelo:

Asegurarse de que la quantización no ha reducido significativamente la precisión del modelo.
Comparar las predicciones del modelo original y del modelo quantizado en un conjunto de datos de prueba.
2.3. Implementar Asynchronous Processing (Opcional)
Si la API recibe una gran cantidad de solicitudes simultáneas, procesarlas de manera asíncrona puede mejorar la capacidad de respuesta.

Instalar Celery y Redis como Broker:

bash
Copiar
pip install celery
Configurar Celery en api.py:

python
Copiar
# api.py (añadir configuración de Celery)

from celery import Celery

# Configurar Celery
celery = Celery(
    app.name,
    broker=redis_url,
    backend=redis_url
)

@celery.task
def predict_task(file_bytes):
    # Procesamiento de la imagen y predicción (similar a predict_web)
    # Devuelve el resultado de la predicción
    # ...
    return {'prediccion': pred_class, 'probabilidad': pred_prob}

@app.route('/predict_async', methods=['POST'])
@require_api_key
def predict_async():
    if 'file' not in request.files:
        return jsonify({'error': 'No se encontró el archivo en la solicitud.'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No se seleccionó ningún archivo.'}), 400
    if not allowed_file(file.filename):
        return jsonify({'error': 'Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF.'}), 400
    try:
        file_bytes = file.read()
        task = predict_task.delay(file_bytes)
        return jsonify({'task_id': task.id}), 202
    except Exception as e:
        return jsonify({'error': str(e)}), 500
Ejecutar el Worker de Celery:

bash
Copiar
celery -A api.celery worker --loglevel=info
Crear un Endpoint para Consultar el Estado de la Tarea:

python
Copiar
@app.route('/tasks/<task_id>', methods=['GET'])
@require_api_key
def get_task_status(task_id):
    task = predict_task.AsyncResult(task_id)
    if task.state == 'PENDING':
        response = {
            'state': task.state,
            'status': 'Pendiente...'
        }
    elif task.state != 'FAILURE':
        response = {
            'state': task.state,
            'result': task.result
        }
    else:
        response = {
            'state': task.state,
            'status': str(task.info)  # Información sobre el error
        }
    return jsonify(response)
Nota: La implementación asíncrona es opcional y depende de las necesidades de tu aplicación. Puede ser útil si esperas una alta concurrencia o tareas de predicción que tomen mucho tiempo.

3. Configurar la Escalabilidad en Heroku
A medida que aumenta el tráfico a tu API, es crucial que la aplicación pueda escalar para manejar la carga sin degradar el rendimiento.

3.1. Escalar Dynos en Heroku
Los dynos son las unidades de ejecución en Heroku. Escalar dynos permite manejar más solicitudes simultáneas.

Escalar Dynos Manualmente:

bash
Copiar
heroku ps:scale web=2 --app mnist-flask-api
Explicación:

Este comando aumenta el número de dynos web a 2, permitiendo que la aplicación maneje más tráfico.
Verificar el Número de Dynos:

bash
Copiar
heroku ps --app mnist-flask-api
3.2. Configurar Auto-Scaling (si es aplicable)
Heroku ofrece planes con capacidades de auto-scaling que ajustan automáticamente el número de dynos según la carga.

Actualizar el Plan de Dynos:
Cambia a un plan que soporte auto-scaling, como el plan Performance.
Configurar Auto-Scaling:
En el dashboard de Heroku, ve a la sección de Resources.
Configura las reglas de auto-scaling según las métricas de uso (CPU, memoria, etc.).
Nota: El auto-scaling está disponible en ciertos planes de Heroku y puede incurrir en costos adicionales.

4. Mejorar la Gestión de Logs y Monitoreo
Mantener una buena gestión de logs y un monitoreo efectivo es esencial para detectar y resolver problemas rápidamente.

4.1. Refinar el Uso de Loguru
Asegúrate de que los logs sean claros y útiles para el diagnóstico.

Configurar Niveles de Logging:
Usa diferentes niveles (INFO, WARNING, ERROR) para categorizar los mensajes.
Integrar Loguru con Servicios de Logs Externos:
Considera enviar logs a servicios como Papertrail, Loggly o ELK Stack para una mejor gestión y análisis.
4.2. Mejorar el Monitoreo con Prometheus y Grafana
Refina las métricas monitoreadas para obtener una visión más detallada del rendimiento de la API.

Añadir Métricas Personalizadas:
Monitorea métricas como el tiempo de respuesta por endpoint, tasa de errores, etc.
Crear Dashboards Personalizados en Grafana:
Diseña dashboards que resalten las métricas más importantes para tu aplicación.
Configurar Alertas:
Configura alertas para notificarte cuando ciertas métricas superen umbrales críticos (e.g., alta latencia, aumento en la tasa de errores).
4.3. Utilizar Herramientas de Monitoreo Adicionales (Opcional)
Considera integrar otras herramientas de monitoreo para obtener una visión más completa, como:

New Relic: Para monitoreo de rendimiento de aplicaciones.
Sentry: Para seguimiento de errores en tiempo real.
5. Conclusiones y Recomendaciones
5.1. Optimización del Rendimiento
Caching con Redis: Reduce la carga del servidor y disminuye la latencia al almacenar resultados de predicciones frecuentes.
Optimización del Modelo: La quantización y otras técnicas de optimización mejoran los tiempos de inferencia sin comprometer significativamente la precisión.
5.2. Escalabilidad en Heroku
Escalado de Dynos: Permite manejar incrementos en el tráfico ajustando el número de dynos según la demanda.
Auto-Scaling: Automatiza el proceso de escalado, asegurando que la aplicación se mantenga receptiva bajo cargas variables.
5.3. Gestión de Logs y Monitoreo
Logs Claros y Detallados: Facilitan la identificación y resolución de problemas.
Monitoreo Avanzado: Proporciona visibilidad en tiempo real del rendimiento de la API, permitiendo acciones proactivas ante posibles incidencias.
Recomendaciones Adicionales
Continuar Mejorando la Seguridad:
Implementar autenticación más robusta como OAuth2 o JWT.
Añadir rate limiting para prevenir abusos.
Ampliar la Funcionalidad de la API:
Añadir endpoints adicionales para diferentes tipos de predicciones o análisis.
Optimizar el Pipeline de CI/CD:
Incorporar etapas adicionales como análisis estático de código, pruebas de integración o despliegue en múltiples entornos.
Automatizar la Actualización del Modelo:
Crear flujos de trabajo para actualizar el modelo de machine learning con nuevos datos automáticamente, asegurando que la API siempre utilice el modelo más actualizado y preciso.
6. Recursos Adicionales
Redis Documentation: Redis Docs
Celery Documentation: Celery Docs
PyTest Documentation: PyTest Docs
GitHub Actions Documentation: GitHub Actions Docs
Heroku Scaling Dynos: Heroku Scaling Dynos
Loguru Documentation: Loguru Docs
Prometheus Documentation: Prometheus Docs
Grafana Documentation: Grafana Docs
TensorFlow Lite Model Optimization: TensorFlow Lite Optimization
Celery with Flask: Celery Flask Integration
Conclusión
En el Día 22, has fortalecido significativamente la eficiencia y capacidad de escalado de tu API de predicción mediante la implementación de caching con Redis, la optimización del modelo de machine learning y la configuración de la escalabilidad en Heroku. Además, has mejorado la gestión de logs y el monitoreo, asegurando que tu aplicación pueda manejar incrementos en la demanda mientras mantiene un rendimiento óptimo.

Pasos Clave Realizados:

Optimización del Rendimiento:

Implementaste caching con Redis para reducir la latencia y la carga del servidor.
Optimizar el modelo mediante quantización para mejorar los tiempos de inferencia.
Configuración de la Escalabilidad:

Escalaste manualmente los dynos en Heroku para manejar mayor tráfico.
Configuraste auto-scaling (si es aplicable) para ajustar automáticamente los recursos según la demanda.
Mejora de Logs y Monitoreo:

Refinaste el uso de Loguru para una mejor gestión de logs.
Mejoraste el monitoreo con Prometheus y Grafana para una visión más detallada del rendimiento de la API.
Recomendaciones para Continuar:

Continuar Mejorando la Seguridad: Implementa métodos de autenticación más robustos y añade mecanismos de prevención contra abusos.
Ampliar la Funcionalidad: Añade más endpoints y funcionalidades a tu API para ampliar su utilidad.
Optimizar el Pipeline de CI/CD: Incorpora etapas adicionales y automatiza más aspectos del desarrollo y despliegue.
Automatizar la Actualización del Modelo: Implementa flujos de trabajo para mantener tu modelo actualizado con nuevos datos.